### Exercise 3: Classification of Airline Tweets with Pretrained Transformers

##### Import the libraries necessary for this project.

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

##### Data Preprocessing: Loading the dataset and preprocessing steps as in Exercise 1.

In [2]:
def remove_unwanted(text):
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    emojis = re.compile(
        "[\U0001F600-\U0001F64F" # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
        "\U0001F700-\U0001F77F"  # Alchemical Symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"  # Enclosed characters
        "]", flags=re.UNICODE
    )
    text = emojis.sub(r'', text)
    return text

def preprocessing(sentence):
    sentence = remove_unwanted(sentence)
    sentence = sentence.lower()
    tokens = word_tokenize(sentence, language='english', preserve_line=True)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    text = " ".join(filtered_tokens)
    return filtered_tokens

In [3]:
def load_data(data_file):
    df = pd.read_csv(data_file, encoding="utf-8")
    tokens = [preprocessing(sentence) for sentence in df['text']]
    df['text'] = [" ".join(token) for token in tokens]
    texts = df['text'].tolist()
    labels = [1 if sentiment == "positive" else 0 for sentiment in df['airline_sentiment'].tolist()]
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(df['airline_sentiment'].tolist())
    label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    print(f'The labels of the dataset are: {label_mapping}')
    return texts, labels

In [4]:
texts, labels = load_data("datasets/Tweets.csv")

The labels of the dataset are: {'negative': 0, 'neutral': 1, 'positive': 2}


In [5]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    return "positive" if preds.item() == 2 else ("neutral" if preds.item() == 1 else "negative")

In [6]:
# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 3
max_length = 128
batch_size = 16
num_epochs = 4
learning_rate = 2e-5

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

bert_model = BERTClassifier(bert_model_name, num_classes)

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Move your model to the selected device
model = bert_model.to(device)

optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

Using device: cpu


In [7]:
# Updated loss function with weights
loss_function = torch.nn.BCEWithLogitsLoss()

# Initialize a display object for updating output in-place
display_id = 'batch_update'
display_obj = display("", display_id=display_id)

# Initialize lists to store history
train_losses = []
train_f1_scores = []
val_losses = []
val_f1_scores = []
patience = 0

# Initialize variables to track best validation loss and corresponding model weights
best_val_loss = float('inf')
best_val_f1 = float('-inf')
best_model_weights = None

''

In [ ]:
for epoch in range(num_epochs):
     print(f"Epoch {epoch + 1}/{num_epochs}")
     train(model, train_dataloader, optimizer, scheduler, device)
     accuracy, report = evaluate(model, val_dataloader, device)
     print(f"Validation Accuracy: {accuracy:.4f}")
     print(report)

torch.save(model.state_dict(), "bert_classifier.pth")

# Test sentiment prediction
test_text = "The movie was great and I really enjoyed the performances of the actors."
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print("The movie was great and I really enjoyed the performances of the actors.")
print(f"Predicted sentiment: {sentiment}")

# Test sentiment prediction
test_text = "Worst movie of the year."
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print("Worst movie of the year.")
print(f"Predicted sentiment: {sentiment}")

Epoch 1/4


In [ ]:
def plot_history(train_losses, train_f1_scores, val_losses, val_f1_scores):
    """
    Plot the history of epochs for loss and accuracy
    :param train_losses: List of training losses for each epoch
    :param train_f1_scores: List of training f1 scores for each epoch
    :param val_losses: List of validation losses for each epoch
    :param val_f1_scores: List of validation f1 scores for each epoch
    :return: None
    """
    epochs = range(1, len(train_losses) + 1)

    # Plot training and validation losses
    plt.plot(epochs, train_losses, 'b', label='Training loss')
    plt.plot(epochs, val_losses, 'r', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Plot training and validation f1 scores
    plt.plot(epochs,train_f1_scores, 'b', label='Training f1-score')
    plt.plot(epochs, val_f1_scores, 'r', label='Validation f1-score')
    plt.title('Training and Validation =f1-scores')
    plt.xlabel('Epochs')
    plt.ylabel('f1_score')
    plt.legend()
    plt.show()

history_dict = {
    'train_losses': train_losses,
    'train_f1_scores': train_f1_scores,
    'val_losses': val_losses,
    'f1_scores': val_f1_scores,
}

history_df = pd.DataFrame(history_dict)
history_df.to_csv('datasets/history_results.csv', index=False)
plot_history(train_losses, train_f1_scores, val_losses, val_f1_scores)

In [ ]:
# Step 11: Confusion Matrix
# Predict on test data
predictions = trainer.predict(test_data)
pred_labels = predictions.predictions.argmax(axis=-1)
true_labels = test_data['label']

# Generate confusion matrix
cm = confusion_matrix(true_labels, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()